# Editing SQL Database

Sound files are identified by instrument + player + klass

Example: klass: 'good-sounds' = 'reference' folder

Sound files are named in the format: instrument_player_klasscategory

### Connecting to database and cleaning up

In [17]:
import sqlite3 
sqliteConnection = sqlite3.connect('database.sqlite')
cursor = sqliteConnection.cursor()

# Test connection
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor.execute(sql_query)
print(cursor.fetchall())

[('packs',), ('takes',), ('ratings',), ('sounds',), ('sounds_emptyklass',), ('violinSounds',)]


In [23]:
%pip install pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 14.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd

sql_query = """PRAGMA table_info(sounds);"""
cursor.execute(sql_query)

# Fetch the results
columns_info = cursor.fetchall()

# Define column names
columns = ["cid", "name", "type", "notnull", "dflt_value", "pk"]

# Create a pandas DataFrame
df = pd.DataFrame(columns_info, columns=columns)

# Export the DataFrame to an Excel file
df.to_excel("table_info.xlsx", index=False, engine="openpyxl")

print("Table exported to 'table_info.xlsx'")


Table exported to 'table_info.xlsx'


In [6]:
#filter out violin instrument into a new table
query = """CREATE TABLE violinSounds AS 
            SELECT * FROM sounds WHERE instrument = 'violin'"""
cursor.execute(query)
#1383 rows of data
sqliteConnection.commit()

In [9]:
#display all folder where the audio belongs to
query = """SELECT DISTINCT folder FROM violinSounds"""
cursor.execute(query)
print(cursor.fetchall())

[('violin_raquel_reference',), ('violin_raquel_dynamics_stability',), ('violin_raquel_pitch_stability',), ('violin_raquel_timbre_stability',), ('violin_raquel_richness',), ('violin_raquel_attack',), ('violin_laia_improvement_recordings',), ('violin_laia_improvement_recordings_2',), ('violin_violin_scales_laia_recordings',)]


In [8]:
#display all different types of klasses the dataset has
query = """SELECT DISTINCT klass FROM violinSounds"""
cursor.execute(query)
print(cursor.fetchall())

[('good-sound',), ('bad-dynamics-crescendo',), ('bad-dynamics-decrescendo',), ('bad-dynamics-tremolo',), ('bad-dynamics-errors',), ('bad-pitch-vibrato',), ('bad-pitch-errors',), ('bad-timbre-errors',), ('bad-richness-bridge',), ('bad-richness-sultasto',), ('bad-attack-pressure',), ('bad-attack-rebond',), ('bad-dynamics',), ('bad-pitch',), ('bad-timbre',), ('bad-richness',), ('bad-attack',), ('scale-good-staccato',), ('scale-good-staccato-minor',), ('scale-bad-pitch-staccato',), ('scale-bad-pitch-staccato-minor',), ('scale-bad-rhythm-staccato',), ('scale-bad-rhythm-staccato-minor',), ('scale-bad-attack-staccato',), ('scale-bad-attack-staccato-minor',), ('scale-bad-timbre-staccato',), ('scale-bad-timbre-staccato-minor',), ('scale-bad-pitch-dirt-staccato',), ('scale-bad-pitch-dirt-staccato-minor',)]


In [2]:
#Number of rows
query = """SELECT * FROM violinSounds"""
cursor.execute(query)
print(len(cursor.fetchall()))

1383


In [25]:
#All the possible klasses 

klasses = [('good-sound'), 
('bad-dynamics-crescendo'), 
('bad-dynamics-decrescendo'), 
('bad-dynamics-tremolo'), 
('bad-dynamics-errors'), 
('bad-pitch-vibrato'), 
('bad-pitch-errors'), 
('bad-timbre-errors'), 
('bad-richness-bridge'), 
('bad-richness-sultasto'), 
('bad-attack-pressure'), 
('bad-attack-rebond'), 
('bad-dynamics'), 
('bad-pitch'), 
('bad-timbre'), 
('bad-richness'), 
('bad-attack'), 
('scale-good-staccato'),
('scale-good-staccato-minor'), 
('scale-bad-pitch-staccato'), 
('scale-bad-pitch-staccato-minor'), 
('scale-bad-rhythm-staccato'), 
('scale-bad-rhythm-staccato-minor'), 
('scale-bad-attack-staccato'), 
('scale-bad-attack-staccato-minor'), 
('scale-bad-timbre-staccato'), 
('scale-bad-timbre-staccato-minor'), 
('scale-bad-pitch-dirt-staccato'), 
('scale-bad-pitch-dirt-staccato-minor')]

print(len(klasses))


29


## Insights

Some audio have multiple klasses sewn into 1. For example, 'bad-richness bad-timbre bad-pitch' is considered as a type of klass in the dataset but it is infact 3 klasses: 'bad-richness', 'bad-timbre' and 'bad-pitch'. 
Thus, the relationship between audio and klasses is 1-m.

In this case, it is a Multi-label classification problem in deep learning. So, when pre-processing the dataset, please follow the labeling techniques of Multi-label classification. For example, suppose there are 20 klasses, where 'bad-richness', 'bad-timbre' and 'bad-pitch' are klass 0, 1, and 2. For a music piece that belongs to these three klasses, it should be labeled as a 20-dimensional vector (1, 1, 1, 0, 0, ..., 0).

Need to label each existing class.### Create json version of database (local)

In [48]:
import sqlite3 
sqliteConnection = sqlite3.connect('database.sqlite')
cursor = sqliteConnection.cursor()

query = "ALTER TABLE violinSounds DROP COLUMN labelvector"
cursor.execute(query)

In [49]:
#Labelling each row. NO NEED TO RUN AGAIN

klass_arr = ['good-sound', 'crescendo', 'decrescendo', 'tremolo', 'vibrato', 'errors', 'bad-pitch', 'bad-dynamics',
             'bad-timbre','bad-richness', 'bad-attack', 'bad-rhythm', 'bridge', 'sultasto', 'pressure', 'rebond', 
             'scale-good', 'staccato', 'minor', 'dirt']

#new column 'labelvector' created to hold label of 20 klasses
#eg. audio is 'klass': 'bad-richness-bridge', vector will be (0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0)

#populate the column with their vectors
import sqlite3 
sqliteConnection = sqlite3.connect('database.sqlite')
cursor = sqliteConnection.cursor()

# cursor.execute('ALTER TABLE violinSounds ADD labelvector varchar(255)')  #add new column
# sqliteConnection.commit()
data = cursor.execute('SELECT * FROM violinSounds').fetchall()

alter_table_query = "ALTER TABLE violinSounds ADD COLUMN labelvector TEXT"
cursor.execute(alter_table_query)

for row in data:
    vec = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    ref = row[21]
    ID = row[0]
    for i in range(len(klass_arr)):
        if klass_arr[i] == 'crescendo':        #decrescendo contain crescendo
            if 'decrescendo' in ref:
                vec[i] = 0
            elif klass_arr[i] in ref:
                vec[i] = 1
        elif klass_arr[i] in ref:
            vec[i] = 1
            
    cursor.execute('UPDATE violinSounds SET labelvector = ? WHERE id = ?',(str(tuple(vec)), ID))
    sqliteConnection.commit()

sqliteConnection.close()

#data is now labelled

In [50]:
sqliteConnection.close()

### Create json version of database (local)

In [ ]:
# format: 
# {
#     "wav": "path to audio",
#     "labels": "label_id1, label_id2, label_id3"
# }

In [51]:
import sqlite3 
sqliteConnection = sqlite3.connect('database.sqlite')
cursor = sqliteConnection.cursor()

# Test connection
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor.execute(sql_query)
print(cursor.fetchall())

[('packs',), ('takes',), ('ratings',), ('sounds',), ('sounds_emptyklass',), ('violinSounds',)]


In [52]:
import json
query = "SELECT folder, pack_filename, labelvector FROM violinSounds"
cursor.execute(query)
rows = cursor.fetchall()

base_dir = r"C:\Users\agnes\Documents\NTU\z_Y4S1 Modules\FYP\Train Dataset\good-sounds\sound_files"

data_list = []
for folder, pack_filename, labelvector in rows:
    file_path = f"{base_dir}\{folder}\{pack_filename}"
    #print(file_path)
    labelvector = eval(labelvector)
    label_ids = [str(index + 1) for index, value in enumerate(labelvector) if value == 1]

    data_list.append({
        "wav": file_path,
        "labels": ", ".join(label_ids)
    })

output_data = {
    "data": data_list
}

# Save to a JSON file
output_json_path = "data.json"
with open(output_json_path, "w") as json_file:
    json.dump(output_data, json_file, indent=4)

print(f"JSON file created at: {output_json_path}")

JSON file created at: data.json


### Create json version of database (colab)

In [53]:
import json
query = "SELECT folder, pack_filename, labelvector FROM violinSounds"
cursor.execute(query)
rows = cursor.fetchall()

base_dir = r"sound_files"

data_list = []
for folder, pack_filename, labelvector in rows:
    file_path = f"{base_dir}/{folder}/{pack_filename}"
    #print(file_path)
    labelvector = eval(labelvector)
    label_ids = [str(index + 1) for index, value in enumerate(labelvector) if value == 1]

    data_list.append({
        "wav": file_path,
        "labels": ", ".join(label_ids)
    })

output_data = {
    "data": data_list
}

# Save to a JSON file
output_json_path = "data_colab.json"
with open(output_json_path, "w") as json_file:
    json.dump(output_data, json_file, indent=4)

print(f"JSON file created at: {output_json_path}")

JSON file created at: data_colab.json


In [54]:
sqliteConnection.close()

## Split Dataset in Train-test

In [55]:
import json
import random

# Load the dataset from JSON
with open("data_colab.json", "r") as file:
    data = json.load(file)

# Extract the list of samples
data_items = data["data"]  # Assuming the dataset is stored under "data" key

# Shuffle the dataset to ensure randomness
random.shuffle(data_items)

# Compute split index (80% train, 20% test)
split_idx = int(0.8 * len(data_items))

# Split data
train_data = {"data": data_items[:split_idx]}
test_data = {"data": data_items[split_idx:]}

# Save training set
with open("train_data.json", "w") as train_file:
    json.dump(train_data, train_file, indent=4)

# Save testing set
with open("test_data.json", "w") as test_file:
    json.dump(test_data, test_file, indent=4)

print("Train-test split completed!")


Train-test split completed!


## Number of 1s in each class for Train dataset

In [56]:
import json

# Load the dataset
with open("train_data.json", "r") as file:
    data = json.load(file)

# Assuming the dataset is stored under the "data" key
data_items = data["data"]

# Number of classes
num_classes = 20

# Initialize a list with 20 zeros to store counts
label_counts = [0] * num_classes

# Count occurrences of each label
for item in data_items:
    labels = item["labels"].split(",")  # Convert "2,8" into a list of strings ["2", "8"]
    for label in labels:
        label_index = int(label) - 1  # Convert to zero-based index
        label_counts[label_index] += 1

print("Label counts:", label_counts)


Label counts: [175, 30, 31, 30, 26, 92, 184, 285, 113, 113, 134, 41, 31, 32, 30, 28, 61, 216, 95, 24]
